# Metaxu Development Notebook

This notebook provides an interactive environment for developing and debugging Metaxu code.

In [ ]:
import sys
sys.path.append('../src')

from metaxu import MetaxuCompiler, CompileOptions

In [ ]:
# Create compiler with debug options
options = CompileOptions(dump_ast=True, dump_ir=True)
compiler = MetaxuCompiler(options)

In [ ]:
# Test simple expression
source = """
let x = 42
print(x)
"""

code = compiler.compile_str(source)
result = compiler.run_vm(code)
print("Result:", result)

In [ ]:
# Test algebraic effects
effect_source = """
effect Logger {
    log(message: String) -> Unit
}

fn main() {
    handle {
        Logger.log(msg) -> {
            print(msg)
            resume(())
        }
    } in {
        perform Logger.log("Hello effects!")
    }
}
"""

code = compiler.compile_str(effect_source)
result = compiler.run_vm(code)

In [ ]:
# Test ownership types
ownership_source = """
struct Buffer {
    data: unique [Int]
}

fn process(buf: &mut Buffer) {
    buf.data[0] = 42
}

fn main() {
    let buf = Buffer { data: [1, 2, 3] }
    process(&mut buf)
    print(buf.data[0])
}
"""

code = compiler.compile_str(ownership_source)
result = compiler.run_vm(code)

## Debugging Tools

Helper functions for debugging compiler stages

In [ ]:
def debug_parse(source):
    """Debug parsing stage"""
    ast = compiler.parser.parse(source)
    print("AST:")
    print(ast)
    return ast

def debug_typecheck(ast):
    """Debug type checking stage"""
    compiler.type_checker.check(ast)
    if compiler.type_checker.errors:
        print("Type Errors:")
        for error in compiler.type_checker.errors:
            print(f"- {error}")
    else:
        print("Type checking passed!")

def debug_codegen(ast):
    """Debug code generation stage"""
    code = compiler.code_generator.generate(ast)
    print("Generated Code:")
    print(code)
    return code

In [ ]:
# Example of using debug tools
source = """
effect State<T> {
    get() -> T
    put(value: T) -> Unit
}

fn counter() -> Int {
    perform State.get() + 1
}
"""

ast = debug_parse(source)
debug_typecheck(ast)
code = debug_codegen(ast)